In [1]:
# =========================================
# Predicción de temperatura futura con LSTM
# =========================================

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
import joblib

# 1. Cargar datos correctamente
df = pd.read_csv("dataset_ml.csv", sep=";", decimal=".", encoding="utf-8")

# Quitar espacios extra en nombres de columnas
df.columns = df.columns.str.strip()

print("✅ Columnas detectadas:", df.columns)

# 2. Procesar columna de tiempo si existe
if 'momento' in df.columns:
    df['momento'] = pd.to_datetime(df['momento'], errors='coerce')
    cols = df.columns.drop('momento')
else:
    cols = df.columns

# 3. Convertir a numérico y limpiar valores inválidos
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')
df = df.replace([np.inf, -np.inf], np.nan).dropna()

# Verificar cantidad de filas
print(f"📊 Total de filas luego de limpiar: {len(df)}")

# 4. Escalado de todas las variables de entrada
scaler = StandardScaler()
scaled_data = scaler.fit_transform(df[cols])

# Guardar scaler para uso futuro
joblib.dump(scaler, "scaler_clima.pkl")

# 5. Crear secuencias de tiempo
def crear_secuencias(datos, n_pasos, columna_objetivo):
    X, y = [], []
    for i in range(n_pasos, len(datos)):
        X.append(datos[i - n_pasos:i])
        y.append(datos[i, columna_objetivo])
    return np.array(X), np.array(y)

# Usaremos 24 pasos (por ejemplo, 24 horas si los datos son horarios)
n_pasos = 24
# Buscar el índice de la columna 'ts' (temperatura) en el escalado
columna_objetivo = list(cols).index('ts')

X, y = crear_secuencias(scaled_data, n_pasos, columna_objetivo)

print(f"✅ Secuencias creadas: X={X.shape}, y={y.shape}")

# 6. Dividir en entrenamiento y test
porc_entrenamiento = 0.8
n_train = int(len(X) * porc_entrenamiento)

X_train, X_test = X[:n_train], X[n_train:]
y_train, y_test = y[:n_train], y[n_train:]

# 7. Construcción del modelo LSTM
model = Sequential()
model.add(LSTM(128, return_sequences=True, input_shape=(n_pasos, X.shape[2])))
model.add(Dropout(0.2))
model.add(LSTM(64))
model.add(Dropout(0.2))
model.add(Dense(1))  # salida: temperatura futura

model.compile(optimizer=Adam(learning_rate=0.001), loss='mse', metrics=['mae'])

# 8. Entrenamiento
early_stop = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=100,
    batch_size=32,
    callbacks=[early_stop],
    verbose=1
)

# 9. Evaluación del modelo
loss, mae = model.evaluate(X_test, y_test)
print(f"📉 MAE en test: {mae:.2f} °C")

# 10. Predicción futura usando la última ventana
ultima_secuencia = X[-1].reshape(1, n_pasos, X.shape[2])
prediccion_escalada = model.predict(ultima_secuencia)[0][0]

# 11. Invertir el escalado para obtener la temperatura real
dummy = np.zeros((1, scaled_data.shape[1]))
dummy[0, columna_objetivo] = prediccion_escalada
prediccion_real = scaler.inverse_transform(dummy)[0, columna_objetivo]

print(f"🌡️ Temperatura predicha a futuro: {prediccion_real:.2f} °C")

# 12. Guardar modelo y scaler
model.save("modelo_lstm_clima.h5")
print("✅ Modelo guardado como 'modelo_lstm_clima.h5'")
print("✅ Scaler guardado como 'scaler_clima.pkl'")



✅ Columnas detectadas: Index(['momento', 'ts', 'td', 'tMin12Horas', 'tMax12Horas', 'tMin24Horas',
       'hr', 'p0', 'qfe1', 'qfe2', 'qff', 'qnh', 'tPromedio24h', 'deltaTemp1h',
       'deltaPresion1h', 'humedadRelativaCambio'],
      dtype='object')
📊 Total de filas luego de limpiar: 96566
✅ Secuencias creadas: X=(96542, 24, 15), y=(96542,)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/rnn/rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 24s 8ms/step - loss: 0.0238 - mae: 0.0956 - val_loss: 0.0025 - val_mae: 0.0431
Epoch 2/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0065 - mae: 0.0594 - val_loss: 0.0012 - val_mae: 0.0272
Epoch 3/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 8ms/step - loss: 0.0062 - mae: 0.0575 - val_loss: 5.9358e-04 - val_mae: 0.0195
Epoch 4/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0059 - mae: 0.0560 - val_loss: 0.0017 - val_mae: 0.0325
Epoch 5/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.0057 - mae: 0.0548 - val_loss: 9.9663e-04 - val_mae: 0.0248
Epoch 6/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 19s 8ms/step - loss: 0.0055 - mae: 0.0540 - val_loss: 5.3553e-04 - val_mae: 0.0182
Epoch 7/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 21s 9ms/step - loss: 0.0054 - mae: 0.0531 - val_loss: 0.0012 - val_mae: 0.0295
Epoch 8/100
2414/2414 ━━━━━━━━━━━━━━━━━━━━ 20s 8ms/step - loss: 0.0055 - mae: 0.0534 - val_loss: 8.5169e-04 - val_mae: 0.0223
Epoch 9/

🌡️ Temperatura predicha a futuro: 14.09 °C
✅ Modelo guardado como 'modelo_lstm_clima.h5'
✅ Scaler guardado como 'scaler_clima.pkl'
